In [1]:
from fastai.vision.all import *
import fastai
from wwf.vision.timm import *
import timm

In [2]:
!nvidia-smi

Thu Mar 25 12:51:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:19:00.0 Off |                  N/A |
| 28%   35C    P8    24W / 250W |     10MiB / 10989MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:1A:00.0 Off |                  N/A |
| 28%   37C    P8    15W / 250W |     10MiB / 10989MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [3]:
import torch
torch.cuda.set_device(2)

In [4]:
path = Path('datasetCorregido/')
trainPath = path/'train'
testPath = path/'test'

In [19]:
dbTest = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items=get_image_files, 
                 splitter=GrandparentSplitter(valid_name='test'),
                 get_y=parent_label,
                 item_tfms = Resize(512),
                 batch_tfms=[*aug_transforms(size=256, min_scale=0.75), Normalize.from_stats(*imagenet_stats)])
dlsTest = dbTest.dataloaders(path,bs=32)

In [66]:
learn = Learner(dlsTest, timm.models.hrnet.hrnet_w32(num_classes=2),metrics=[accuracy,Precision(),Recall(),RocAucBinary()],loss_func=LabelSmoothingCrossEntropy()).to_fp16()

In [69]:
learn.load('hrnet')

/home/jonathan/.virtualenvs/segmentation/lib/python3.6/site-packages/fastai/learner.py:56: UserWarning: Saved filed doesn't contain an optimizer state.
  elif with_opt: warn("Saved filed doesn't contain an optimizer state.")


In [70]:
learn.validate()

(#5) [0.5184487104415894,0.7881041169166565,0.888283378746594,0.7149122807017544,0.8774147798270605]

In [71]:
preds,y=learn.get_preds()

In [72]:
predIdxs = np.argmax(preds.numpy(), axis=1)

In [73]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(predIdxs, y).ravel()

print(tn / (tn+fp))

0.7045454545454546


## TTA

In [ ]:
preds,y=learn.tta(n=6)

In [ ]:
predIdxs = np.argmax(preds.numpy(), axis=1)

In [ ]:
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score,roc_auc_score,classification_report

In [59]:
precision_score(predIdxs,y)

0.7982456140350878

In [60]:
recall_score(predIdxs,y)

0.8921568627450981

In [61]:
accuracy_score(predIdxs,y)

0.8314745972738538

In [62]:
f1_score(y,predIdxs)

0.8425925925925927

In [63]:
roc_auc_score(y,preds[:,1])

0.9191345529064827

In [ ]:
tn, fp, fn, tp = confusion_matrix(predIdxs, y).ravel()

print(tn / (tn+fp))

In [65]:
print(classification_report(y,predIdxs,digits=4))

              precision    recall  f1-score   support

           0     0.7694    0.8746    0.8187       351
           1     0.8922    0.7982    0.8426       456

    accuracy                         0.8315       807
   macro avg     0.8308    0.8364    0.8306       807
weighted avg     0.8388    0.8315    0.8322       807



-------------------------------------------------------

In [54]:
np.max(preds.numpy(), axis=1)

array([0.9303789 , 0.75445956, 0.7427454 , 0.84579724, 0.73201686,
       0.7989495 , 0.7441814 , 0.96091676, 0.95411706, 0.91908157,
       0.8072547 , 0.6259793 , 0.9041923 , 0.66963094, 0.7862104 ,
       0.89290553, 0.59019667, 0.9470582 , 0.6006665 , 0.8151437 ,
       0.8898027 , 0.83778036, 0.8098404 , 0.6502403 , 0.91925997,
       0.8531942 , 0.8712686 , 0.93743676, 0.9570373 , 0.90081346,
       0.8353411 , 0.97862947, 0.51516104, 0.55372745, 0.92228794,
       0.89999545, 0.74750006, 0.89660573, 0.92137724, 0.69277704,
       0.8428752 , 0.90335345, 0.83016133, 0.8917056 , 0.6386014 ,
       0.8868762 , 0.90812147, 0.7777797 , 0.82237184, 0.90417755,
       0.78263026, 0.84783936, 0.61609584, 0.7010434 , 0.9271331 ,
       0.8991491 , 0.8816315 , 0.7694386 , 0.8795728 , 0.8046267 ,
       0.9270269 , 0.76836264, 0.5777951 , 0.81703633, 0.8965539 ,
       0.75859165, 0.6300471 , 0.8663912 , 0.68358636, 0.8304776 ,
       0.8660434 , 0.73298645, 0.83759606, 0.6564155 , 0.90663

In [55]:
confidentPredictions = []
confidentRealValues = []
for i,x in enumerate(np.max(preds.numpy(), axis=1)):
    if(x>=0.7):
        confidentPredictions.append(predIdxs[i])
        confidentRealValues.append(y[i])

In [56]:
precision_score(confidentPredictions,confidentRealValues)

0.8981481481481481

In [57]:
recall_score(confidentPredictions,confidentRealValues)

0.9478827361563518

In [58]:
f1_score(confidentPredictions,confidentRealValues)

0.9223454833597464

In [59]:
accuracy_score(confidentPredictions,confidentRealValues)

0.9156626506024096

In [60]:
len(confidentPredictions)/len(y)

0.7199504337050805